In [2]:
import pandas as pd
import numpy as np
import os


In [3]:
data = pd.read_csv('data.csv')

In [4]:
data = data.iloc[:, :68]

In [5]:
data.to_csv('data.csv', index=False)
data.replace('FALSE', 0, inplace=True)

In [11]:
columns_to_select = [
    'Team Number', 'AC1', 'AC2', 'AC3', 
    'AC4', 'AARL', 'AARH', 'AAP', 'AAB', 'AM', 'TC1', 
    'TC2', 'TC3', 'TC4', 'TARL', 'TARH', 'TAP', 'TAB'
]

data = data[columns_to_select]

In [22]:
averaged_data = data.groupby("Team Number").mean().reset_index()

In [33]:
# Game data for team 1

game1 = [6217,6318,8516]

game1mask = averaged_data.apply(lambda row: any(
    (row['Team Number'] == team)
    for team in game1
), axis=1)

game1data = averaged_data[game1mask]

# Game data for team 2

game2 = [10490,4011,3206]

game2mask = averaged_data.apply(lambda row: any(
    (row['Team Number'] == team)
    for team in game2
), axis=1)

game2data = averaged_data[game2mask]


In [34]:
game1data.sum()

Team Number    21051.000000
AC1                0.111111
AC2                0.000000
AC3                0.000000
AC4                0.000000
AARL               0.000000
AARH               0.000000
AAP                0.000000
AAB                0.000000
AM                 2.666667
TC1                0.888889
TC2                2.444444
TC3                4.333333
TC4                0.444444
TARL               0.888889
TARH               0.666667
TAP                0.555556
TAB                0.000000
dtype: float64

In [35]:
game2data.sum()

Team Number    17707.000000
AC1                0.861111
AC2                0.000000
AC3                0.000000
AC4                0.000000
AARL               0.000000
AARH               0.000000
AAP                0.000000
AAB                0.000000
AM                 2.277778
TC1                4.319444
TC2                1.111111
TC3                2.013889
TC4                0.347222
TARL               0.333333
TARH               0.555556
TAP                3.125000
TAB                0.444444
dtype: float64